**извлечение признаков из текста на естественном языке**

классификатор текстов

частотный анализ, понижаем значение признака для частых слов (IDF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
1904,"Астрономы сообщили, что осадки на экзопланете HAT-P-7b выпадают в виде кристаллов рубинов и сапфиров.\n\nУченые находили планеты с облаками из экзотических материалов\n\nКак сообщают СМИ, речь иде...",science
3183,РАГС (rags.ru) На полку землепользователям Международная школа управления\n?Интенсив? РАГС выпустила в свет сборник ?Землепользование в местах проживания\nкоренных малочисленных народов России: за...,politics


##  CountVectorizer + TF-IDF

In [6]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    return ' '.join( [ t.strip() for t in tt if t ] )
    

In [7]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# понижаем значение признака для частых слов (IDF)
tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=True,norm='l2')
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f1b99ae3400>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [8]:
# размер словаря
len(tf.vocabulary_)

85058

## формируем датасеты

In [9]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 85058)

In [10]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [11]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [12]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [13]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 85058), (319,), (2877, 85058), (2877,))

## обучаем

In [14]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [15]:
o = clf.predict(X_train)

In [16]:
accuracy_score(y_train,o)

1.0

In [17]:
# print( classification_report(y_train,o) )

---

In [18]:
o = clf.predict(X_test)

In [19]:
accuracy_score(y_test,o)

0.7646854362182829

In [20]:
print( classification_report(y_test,o) )

              precision    recall  f1-score   support

           0       0.92      0.77      0.84       222
           1       0.69      0.77      0.73       326
           2       0.59      0.83      0.69       234
           3       0.88      0.08      0.15        86
           4       0.80      0.95      0.87       393
           5       0.79      0.89      0.83       547
           6       0.93      0.25      0.39        57
           7       0.00      0.00      0.00        50
           8       0.82      0.91      0.86       200
           9       0.69      0.08      0.14       137
          10       0.93      0.89      0.91       341
          11       0.63      0.75      0.69       252
          12       0.60      0.47      0.53        32

   micro avg       0.76      0.76      0.76      2877
   macro avg       0.71      0.59      0.59      2877
weighted avg       0.76      0.76      0.73      2877



/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---

In [29]:
# from matplotlib import pyplot as plt
# import itertools

# plt.figure(figsize=(10,9))

# cm = confusion_matrix(y_test,o)
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion matrix')
# plt.colorbar()

# classes = sorted(labels.keys())

# tick_marks = np.arange(len(classes))
# plt.xticks(tick_marks, classes, rotation=45)
# plt.yticks(tick_marks, classes)

# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], 'd'),
#              horizontalalignment="center",
#              color="white" if cm[i, j] > thresh else "black")

# plt.tight_layout()
# plt.ylabel('True label')
# plt.xlabel('Predicted label')

# plt.show()

---

In [22]:
o = clf.predict(X)

In [23]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [24]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: realty
predict: realty
- - - - - - - - - - - - - - - - - - 

Без посольства Турции и домов для многодетных. Какая судьба ждет район Орловской-Нововиленской

1 декабря 2016 в 16:08

Майя Кохно / REALTY.TUT.BY

В среду, 30 ноября, в актовом зале администрации Центрального района прошла презентация проекта детального планирования территории в границах Орловской — Старовиленского тракта и переулка Нововиленского. Общая площадь участка проектирования от посольства России до Орловской составляет чуть более 16 гектаров. Как изменится район, движение транспорта и когда снесут старые дома — в репортаже TUT.BY

До 2020 года — новые жилые дома и паркинги, после 2030 — снос старых домов

Вечером в зале для обсуждения была занята примерно половина мест. Поначалу жители района хотели выступить сразу с вопросами, но все же выслушали презентацию главного архитектора проекта Ирины Обухович.

Территория проектирования разделена условно на 2 квартала — западный и восточный. Застройка западного кварт

---

In [25]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(data['tag'])
# print( list(le.classes_) )
# y = le.transform(data['tag']) 

In [26]:
# from sklearn.pipeline import Pipeline
# text_clf = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', SGDClassifier(loss='hinge')),
#                 ])
# text_clf.fit()

In [27]:
# from sklearn.model_selection import GridSearchCV

---

In [28]:
# Sebastian Raschka   Python Machine Learning  - Packt Publishing Ltd, 2015